In [2]:
import cv2
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [6]:
def shape_transform(h, w):
    img_h_new, img_w_new = int(np.ceil(h / 32) * 32), int(np.ceil(w / 32) * 32)
    scale_h, scale_w = img_h_new / h, img_w_new / w
    return img_h_new, img_w_new, scale_h, scale_w

def postprocess(img_h_new, img_w_new, heatmap, offset, scale, threshold):
    NoneType = type(None)
    dets= decode(heatmap, scale, offset, (img_h_new, img_w_new), threshold=threshold)
    if type(dets) == NoneType:
        dets = np.empty(shape=[0, 5], dtype=np.float32)
    elif len(dets) > 0:
        dets[:, 0:4:2], dets[:, 1:4:2] = dets[:, 0:4:2] / scale_w, dets[:, 1:4:2] / scale_h
    else:
        dets = np.empty(shape=[0, 5], dtype=np.float32)
    return dets

def decode(heatmap, scale, offset,size, threshold=0.1):
    heatmap = np.squeeze(heatmap)
    scale0, scale1 = scale[0, 0, :, :], scale[0, 1, :, :]
    offset0, offset1 = offset[0, 0, :, :], offset[0, 1, :, :]
    c0, c1 = np.where(heatmap > threshold)
    boxes = []
    if len(c0) > 0:
        for i in range(len(c0)):
            s0, s1 = np.exp(scale0[c0[i], c1[i]]) * 4, np.exp(scale1[c0[i], c1[i]]) * 4
            o0, o1 = offset0[c0[i], c1[i]], offset1[c0[i], c1[i]]
            s = heatmap[c0[i], c1[i]]
            x1, y1 = max(0, (c1[i] + o1 + 0.5) * 4 - s1 / 2), max(0, (c0[i] + o0 + 0.5) * 4 - s0 / 2)
            x1, y1 = min(x1, size[1]), min(y1, size[0])
            boxes.append([x1, y1, min(x1 + s1, size[1]), min(y1 + s0, size[0]), s])
        boxes = np.asarray(boxes, dtype=np.float32)
        keep = nms(boxes[:, :4], boxes[:, 4], 0.3)
        boxes = boxes[keep, :]
        return boxes

def nms(boxes, scores, nms_thresh):
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = np.argsort(scores)[::-1]
    num_detections = boxes.shape[0]
    suppressed = np.zeros((num_detections,), dtype=np.bool_)

    keep = []
    for _i in range(num_detections):
        i = order[_i]
        if suppressed[i]:
            continue
        keep.append(i)

        ix1 = x1[i]
        iy1 = y1[i]
        ix2 = x2[i]
        iy2 = y2[i]
        iarea = areas[i]

        for _j in range(_i + 1, num_detections):
            j = order[_j]
            if suppressed[j]:
                continue

            xx1 = max(ix1, x1[j])
            yy1 = max(iy1, y1[j])
            xx2 = min(ix2, x2[j])
            yy2 = min(iy2, y2[j])
            w = max(0, xx2 - xx1 + 1)
            h = max(0, yy2 - yy1 + 1)

            inter = w * h
            ovr = inter / (iarea + areas[j] - inter)
            if ovr >= nms_thresh:
                suppressed[j] = True

    return keep


In [7]:
onnx_path = "centerface.onnx"
face_net = cv2.dnn.readNetFromONNX(onnx_path)

In [8]:
# img = cv2.imread("city.jpg")
cap = cv2.VideoCapture('sample.mp4')
while True:
    ret,img = cap.read() 
    h,w = img.shape[:2]
    # Compute sizes
    h_new, w_new, scale_h, scale_w = shape_transform(h,w)
    blob = cv2.dnn.blobFromImage(
                img, scalefactor=1.0, size=(w_new, h_new),
                mean=(0, 0, 0), swapRB=False, crop=False
            )
    face_net.setInput(blob)
    heatmap, scale, offset = face_net.forward(["537", "538", "539"])
    threshold = 0.2
    detections = postprocess(h_new, w_new,heatmap, offset, scale, threshold)
    for det in detections:
        boxes, score = det[:4], det[4]
        img[int(boxes[1]): int(boxes[3]), int(boxes[0]): int(boxes[2])] = cv2.blur(img[int(boxes[1]): int(boxes[3]), int(boxes[0]): int(boxes[2])], (30,30))
    cv2.imshow('out', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()